In [1]:
# MNIST Handwritten Digits Classification
# Standard feedfoward neural network

# Dataset from: https://www.kaggle.com/oddrationale/mnist-in-csv

In [2]:
import numpy as np
import pandas as pd
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

np.set_printoptions(threshold=100)

In [3]:
# データの読み込み（MNIST手書き文字）
train_df = pd.read_csv('data/mnist-in-csv/mnist_train.csv', sep=',')
test_df = pd.read_csv('data/mnist-in-csv/mnist_test.csv', sep=',')

train_data = train_df.iloc[:,1:].to_numpy(dtype='float')
train_target = train_df.iloc[:,0].to_numpy(dtype='int')
train_target = OneHotEncoder(sparse=False).fit_transform(train_target.reshape(-1, 1))

test_data = test_df.iloc[:,1:].to_numpy(dtype='float')
test_target = test_df.iloc[:,0].to_numpy(dtype='int')
test_target = OneHotEncoder(sparse=False).fit_transform(test_target.reshape(-1, 1))

/home/mech-user/miniconda3/envs/NESTenv/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mech-user/miniconda3/envs/NESTenv/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, y

In [4]:
# 重要パラメータの定義
# データの性質関連
TRAIN_SIZE = 2000        # 訓練データ数
TEST_SIZE = 2000         # テストデータ数
NUM_INPUT_NODES = 784    # 入力の次元数
NUM_OUTPUT_NODES = 10    # 出力の次元数

LEARNING_RATE = 0.01     # 学習率

In [5]:
train_data = train_data[:TRAIN_SIZE]
test_data = test_data[:TEST_SIZE]
train_target = train_target[:TRAIN_SIZE]
test_target = test_target[:TEST_SIZE]

In [6]:
train_data.shape

(2000, 784)

In [7]:
train_target.shape

(2000, 10)

In [8]:
test_data.shape

(2000, 784)

In [9]:
test_target.shape

(2000, 10)

In [10]:
# 結合層や活性化関数をクラスとして定義

class ReLU:   # Rectified Linear Unit
    def forward(self, u): 
        self.u = u
        return np.maximum(0, u)
    
    def backward(self, dout):
        return dout * (self.u > 0).astype(float)   # uが正である場合、微分は1.0
    
    def update_weights(self, lr=0.1):
        pass


class LinearLayer:   # 線形の全結合層
    def __init__(self, I, O):
        self.I = I
        self.O = O
        self.W = np.random.randn(I,O) / np.sqrt(I)   # 重みの初期化。分散 1/I の正規分布
        self.b = np.zeros(O)
        self.grad_W = np.zeros((I,O))
        self.grad_b = np.zeros(O)
        
    def forward(self, x):
        self.x = x
        u = x @ self.W + self.b
        return u
    
    def backward(self, dout):   # dout shape: (O)
        din = self.W @ dout   # shape: (I)
        self.grad_W = self.x.reshape(self.I,1) @ dout.reshape(1, self.O)   # shape: (I, O)
        self.grad_b = dout    # shape: (O)
        return din
    
    def update_weights(self, lr=0.1):
        self.W = self.W - lr * self.grad_W
        self.b = self.b - lr * self.grad_b
        
    
class Softmax_CrossEntropy:   # 多クラス分類の出力層（softmax関数＋交差エントロピー誤差）
    def forward(self, u):
        self.y = np.exp(u)
        self.y = self.y / np.sum(self.y)
        return self.y
    
    def calculate_error(self, t):   # u（すなわちy）とtの次元数は一致する前提
        self.t = t
        error = -np.sum(t * np.log(self.y))
        return error
    
    def backward(self, dout=1.0):
        return dout*(self.y - self.t)
    
    def update_weights(self, lr=0.1):
        pass
        
        

In [11]:
# データの正規化

# def normalize(data):
#     mean = np.mean(data, axis=1).reshape(-1,1)
#     var = np.var(data, axis=1).reshape(-1,1)
#     return (data - mean) / (np.sqrt(var) + 1e-6)

# train_data = normalize(train_data)
# test_data = normalize(test_data)

train_data = train_data / 255.0
test_data = test_data / 255.0


In [12]:
# ネットワークの定義

l1 = LinearLayer(784, 100)
f1 = ReLU()
l2 = LinearLayer(100, 50)
f2 = ReLU()
l3 = LinearLayer(50, 10)
out = Softmax_CrossEntropy()

layers = [l1, f1, l2, f2, l3, out]



In [13]:
# 訓練

for epoch in range(1000):
    error = 0
    
    for i in range(TRAIN_SIZE):
        x = train_data[i]
        t = train_target[i]
        dout = 1.0

        for layer in layers:
            x = layer.forward(x)
        
        error += layers[-1].calculate_error(t)
        
        for layer in reversed(layers):
            dout = layer.backward(dout)

        for layer in layers:
            layer.update_weights(LEARNING_RATE)
    
    if epoch % 1 == 0:
        print("Epoch no. {}, error is {}".format(epoch, error))
    

Epoch no. 0, error is 1312.3278876922604
Epoch no. 1, error is 529.585379369882
Epoch no. 2, error is 289.4006499375038
Epoch no. 3, error is 151.67031756355152
Epoch no. 4, error is 116.12722263436156
Epoch no. 5, error is 82.90289173667081
Epoch no. 6, error is 99.37513520987089
Epoch no. 7, error is 169.8735913745302
Epoch no. 8, error is 85.13244936192041
Epoch no. 9, error is 83.98817765283253
Epoch no. 10, error is 43.181960326894604
Epoch no. 11, error is 15.600262910499806
Epoch no. 12, error is 4.042136470614744
Epoch no. 13, error is 1.4658321158713952
Epoch no. 14, error is 0.7932672752528017
Epoch no. 15, error is 0.6487264692578282
Epoch no. 16, error is 0.5580666392096094
Epoch no. 17, error is 0.49386858182618054
Epoch no. 18, error is 0.44449909537938365
Epoch no. 19, error is 0.405273644736794
Epoch no. 20, error is 0.3729436888227159
Epoch no. 21, error is 0.34593969928004015
Epoch no. 22, error is 0.32296738207774917
Epoch no. 23, error is 0.3030548466450625
Epoch no

Epoch no. 190, error is 0.028109917865106103
Epoch no. 191, error is 0.027956159229682392
Epoch no. 192, error is 0.027804793228629086
Epoch no. 193, error is 0.027654193542962544
Epoch no. 194, error is 0.027506345150436696
Epoch no. 195, error is 0.02735724657627613
Epoch no. 196, error is 0.027212467596928584
Epoch no. 197, error is 0.02706970571575046
Epoch no. 198, error is 0.02692502144235261
Epoch no. 199, error is 0.026785925856092935
Epoch no. 200, error is 0.026644469702552107
Epoch no. 201, error is 0.02650638390793199
Epoch no. 202, error is 0.026371092008242298
Epoch no. 203, error is 0.026235857715760022
Epoch no. 204, error is 0.026100530941675526
Epoch no. 205, error is 0.0259689221207219
Epoch no. 206, error is 0.025837479823168363
Epoch no. 207, error is 0.02570727064870444
Epoch no. 208, error is 0.025579133826395588
Epoch no. 209, error is 0.02545114476193029
Epoch no. 210, error is 0.025326329305502347
Epoch no. 211, error is 0.0252000071303454
Epoch no. 212, error

Epoch no. 374, error is 0.01394731597370073
Epoch no. 375, error is 0.013909356182727027
Epoch no. 376, error is 0.013871336208297816
Epoch no. 377, error is 0.013833733219625362
Epoch no. 378, error is 0.01379644178418374
Epoch no. 379, error is 0.01375936964390317
Epoch no. 380, error is 0.013722255791374021
Epoch no. 381, error is 0.013685308422691998
Epoch no. 382, error is 0.013648725561727216
Epoch no. 383, error is 0.013612115756299809
Epoch no. 384, error is 0.013575949994012008
Epoch no. 385, error is 0.013540331092525441
Epoch no. 386, error is 0.013503763545063939
Epoch no. 387, error is 0.01346847143101381
Epoch no. 388, error is 0.013432471789924414
Epoch no. 389, error is 0.013397108530454031
Epoch no. 390, error is 0.013362218296461425
Epoch no. 391, error is 0.013327071673722083
Epoch no. 392, error is 0.013292509713275205
Epoch no. 393, error is 0.013257717209518093
Epoch no. 394, error is 0.01322301957429024
Epoch no. 395, error is 0.013189286880249944
Epoch no. 396, 

Epoch no. 557, error is 0.009265660242633098
Epoch no. 558, error is 0.009248726769866256
Epoch no. 559, error is 0.009231478258323728
Epoch no. 560, error is 0.009214734187839187
Epoch no. 561, error is 0.009197877604119211
Epoch no. 562, error is 0.009180817899768375
Epoch no. 563, error is 0.009164173894385457
Epoch no. 564, error is 0.009147351183947182
Epoch no. 565, error is 0.009130930040834609
Epoch no. 566, error is 0.009114167063233208
Epoch no. 567, error is 0.009097721649461177
Epoch no. 568, error is 0.009081232055514674
Epoch no. 569, error is 0.009064855981383468
Epoch no. 570, error is 0.009048461596457625
Epoch no. 571, error is 0.009032277320232443
Epoch no. 572, error is 0.009015900008273141
Epoch no. 573, error is 0.009000008437254368
Epoch no. 574, error is 0.008983680834313005
Epoch no. 575, error is 0.00896758815208337
Epoch no. 576, error is 0.008951800309657483
Epoch no. 577, error is 0.008935747298885835
Epoch no. 578, error is 0.008919881695669504
Epoch no. 5

Epoch no. 740, error is 0.006923228264839856
Epoch no. 741, error is 0.006913541305149166
Epoch no. 742, error is 0.006904205753533043
Epoch no. 743, error is 0.0068945234386242
Epoch no. 744, error is 0.006884990563213388
Epoch no. 745, error is 0.006875553825073665
Epoch no. 746, error is 0.006866088522239837
Epoch no. 747, error is 0.006856781437160479
Epoch no. 748, error is 0.006847228430855496
Epoch no. 749, error is 0.006838023796998735
Epoch no. 750, error is 0.006828659901767346
Epoch no. 751, error is 0.006819317557828036
Epoch no. 752, error is 0.0068100442820233495
Epoch no. 753, error is 0.0068008760005105655
Epoch no. 754, error is 0.006791630299989672
Epoch no. 755, error is 0.00678248183113954
Epoch no. 756, error is 0.006773187467653305
Epoch no. 757, error is 0.00676409406498279
Epoch no. 758, error is 0.006754975527075052
Epoch no. 759, error is 0.006745956984247856
Epoch no. 760, error is 0.006736806271087612
Epoch no. 761, error is 0.006727710257836535
Epoch no. 76

Epoch no. 922, error is 0.005532348474745532
Epoch no. 923, error is 0.005526239794821518
Epoch no. 924, error is 0.0055200835261826125
Epoch no. 925, error is 0.005514079501508291
Epoch no. 926, error is 0.005507893535245598
Epoch no. 927, error is 0.005501887876729711
Epoch no. 928, error is 0.005495776705826262
Epoch no. 929, error is 0.0054897715963848695
Epoch no. 930, error is 0.005483759507882581
Epoch no. 931, error is 0.005477705942661265
Epoch no. 932, error is 0.005471710630676236
Epoch no. 933, error is 0.005465725933574405
Epoch no. 934, error is 0.005459711501937611
Epoch no. 935, error is 0.005453854143370895
Epoch no. 936, error is 0.005447748947651613
Epoch no. 937, error is 0.0054419312777669124
Epoch no. 938, error is 0.005435943890016849
Epoch no. 939, error is 0.005430000292531779
Epoch no. 940, error is 0.005424161150079151
Epoch no. 941, error is 0.0054182487304851885
Epoch no. 942, error is 0.005412337212387729
Epoch no. 943, error is 0.005406542255043606
Epoch 

In [14]:
# 検証

correct_number = 0

for i in range(TEST_SIZE):
    x = test_data[i]
    t = test_target[i]

    for layer in layers:
        x = layer.forward(x)

    predict = np.argmax(x)
    correct_value = np.argmax(t)
    
    if predict == correct_value:
        correct_number += 1
        
print("Accuracy: {}".format(correct_number*1.0/TEST_SIZE))

Accuracy: 0.8885
